# Лекция №6

In [1]:
from torch import nn

__Сегментация__ - определение принадлежности каждого пикселя определенному объекту изображения.

Сегментация - классификация пикселей.

Наивный подход:
- Брать окрестность и получать вероятность принадлежности пикселя объекту:
  - долго работает;
  - не учитывает глобального контекста.

Дополнительные трансформации в задаче сегментации:
- Elastic transform(пространственное искажение изображения);

Другой подход - __кластеризация__(иерархическая, например):
- _энергетический_ подход - каждому пикселю ставятся веса в соответствие, чтобы получить разделение поля на независимые области.

Последоательное применение:
1. Нейросетевая классификация объектов на фотографии;
2. Условно случайные поля уточняли сегментацию объектов.

Для DL сегментации нужны обратные операции свертки и пулинга для сохранения размеров изображения.

__ConvTranspose2d__ - каждый элемент входного изображения умножаем на фильтр(ядро) и заполняем соответствующий фаргмент результата, затем суммируем результаты:
- Проблемы:
  - _Шахматные_ эффекты - при суммировании в некоторых фрагментах больше элементов в сумме, чем в других, поэтому эффект шахматной доски.
  - Предотвратить можно, если применить потом увеличение размеров и свертку.

## Fully convolutional networks

- Применяем свертки до тензоров;
- Затем увеличиваем их размеры;
- Тогда от задачи классификации можно получить черновое предположение о структуре изображения(сегментации).
- Чем больше пробрасываний связей, тем точнее результат сегментации.

## Unet

- Сворачивание результата до тензора;
- Апскейл и связь для сегментации;

In [2]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels,
                kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels,
                kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
            )
    def forward(self, x):
        return self.double_conv(x)
      
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
                                nn.MaxPool2d(2),
                                DoubleConv(in_channels,
                                            out_channels))
    def forward(self, x):
        return self.maxpool_conv(x)

In [3]:
class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    def forward(self, x):
        return self.conv(x)

In [4]:
import torch.nn.functional as F
class UNet(nn.Module):
    def __init__(self, n_channels,
        n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

## Тирамису сеть

- добавляет DB-блоки с прокидыванием связей.

## V-Net

- UNet для 3d-сегментации.

## UNet++

- Добавили промежуточные нейроны(больше полносвязности);
- Ошибка - ошибка классификации и сегментации(совместный подсчет ошибок);

### Другие методы

- расширенные свертки(не 3х3, а 6х6, отмечая каждый 2-й пиксель, как свертку):
  - Использовались в DeepLabv[1-3], где использовались еще и пирамидальные архитектуры, пробрасывания связей и тд

- FastFCN:
  - Свертки + разные dilation rates для сверток.



## High-Res Representation

- Разделение сети на несколько _путей_(на каждом разные размеры изображения) с полносвязными соединениями в местах разделения сети на описанные выше _пути_.

## Механизм attention

- будет рассмотрен позже;
- позволяет добиться лучшего качества сегментации.

## Instance segmentation

- Каждый человек - отдельная маска(в семантической все люди бы отметились одной маской);
- Можно сначала запустить детектор, а затем в каждой рамке сегментировать.

### Mask R-CNN

- Сохраняется CNN+RPN;
- Вместо ROI Pooling используется ROI Align;
- Затем генерация маски и классификация;
- Ошибка - сумма 3-х ошибок(регрессия, классификация, сегментация).

- ROI Align:
  - Используется билинейная интерполяция(по осям для заполнения вырезанных окон);

## PolarMask

- каждому пикселю говорим о границе объекта:
  1. Выбрали пиксель;
  2. Провели лучи из этого пикселя;
  3. Накладываем маску на лучи, сохраняем расстояния до границ;
  4. Эти числа должна научиться предсказывать сеть(границы объекта).

## TensorMask

- для каждой точки предскаызваем маску объекта.

